In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas
import datetime
import os
import sys
module_path = os.path.abspath(os.path.join('../data_pipeline/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import sqlite3
import create_onset_data

In [2]:
inds_dbdir = "/home/bharat/Documents/data/ss_onset_dataset/data/sqlite3/"
sm_db_name = "smu_sml_sme.sqlite"
sm_table_name = "smusmlsme"

In [3]:
dataObj = create_onset_data.OnsetData(delTCutoff=2)
ssBinDF = dataObj.create_output_bins(\
                saveFile="../data/binned_data_extra_latlon.feather",\
                aulDBdir=inds_dbdir, \
                 aulDBName=sm_db_name,\
                 aulTabName=sm_table_name)
# print ssBinDF.head()
# print "-------------"
# print ssBinDF.shape

Splitting the data into train, validation and test
                     bin_0  bin_1    mlat_0  mlat_1     mlt_0  mlt_1  \
1996-03-30 03:41:00      0      0 -1.000000    -1.0 -1.000000   -1.0   
1996-03-30 03:42:00      1      0  0.763333    -1.0  0.998056   -1.0   
1996-03-30 03:43:00      1      0  0.763333    -1.0  0.998056   -1.0   
1996-03-30 03:44:00      1      0  0.763333    -1.0  0.998056   -1.0   
1996-03-30 03:45:00      1      0  0.763333    -1.0  0.998056   -1.0   

                     del_minutes data_label  
1996-03-30 03:41:00        30.95          P  
1996-03-30 03:42:00        29.95          P  
1996-03-30 03:43:00        28.95          P  
1996-03-30 03:44:00        27.95          P  
1996-03-30 03:45:00        26.95          P  


In [4]:
ssBinDF.tail(6000)["data_label"].value_counts()

I    3374
P    2626
Name: data_label, dtype: int64

In [8]:
dataObj = create_onset_data.OnsetData(useSML=True,delTCutoff=2,\
                    smlDateRange=[datetime.datetime(1997,1,1),\
                    datetime.datetime(2007,1,1)], fillTimeRes=10)
ssBinDF = dataObj.create_sml_bins()
print ssBinDF.head()
print "-------------"
print ssBinDF.shape

('Working through year-->', 1997)
('Working through year-->', 1998)
('Working through year-->', 1999)
('Working through year-->', 2000)
('Working through year-->', 2001)
('Working through year-->', 2002)
('Working through year-->', 2003)
('Working through year-->', 2004)
('Working through year-->', 2005)
('Working through year-->', 2006)
('Working through year-->', 2007)
                     bin_0  bin_1  mlat_0  mlat_1  mlt_0  mlt_1  del_minutes  \
1997-01-01 00:00:00      0      0    -1.0    -1.0   -1.0   -1.0         -1.0   
1997-01-01 00:10:00      0      0    -1.0    -1.0   -1.0   -1.0         -1.0   
1997-01-01 00:20:00      0      0    -1.0    -1.0   -1.0   -1.0         -1.0   
1997-01-01 00:30:00      0      0    -1.0    -1.0   -1.0   -1.0         -1.0   
1997-01-01 00:40:00      0      0    -1.0    -1.0   -1.0   -1.0         -1.0   

                    data_label  
1997-01-01 00:00:00          S  
1997-01-01 00:10:00          S  
1997-01-01 00:20:00          S  
1997-01-01 00